In [27]:
import pandas as pd
from pathlib import Path

This script will add a column for all classmates of the students. Classmates are placed in random order as a comma separated string. 

In [28]:
participant_data_path = "./example_data"
participant_input_filename = "test-participant-tokens.csv"

In [29]:
column_dict = {
    "Geen toestemming": "attribute_2 <Permission>",
    "Klas": "attribute_3 <Klas>",
    "Stamnr": "token",
}
classmates_column_name = "attribute_1 <Classmates>"


In [30]:
participant_data_path = Path(participant_data_path)
participant_input_filename = participant_data_path / participant_input_filename
participants_df = pd.read_csv(participant_input_filename)

In [31]:
participants_df = participants_df.rename(columns=column_dict)

In [32]:
for i, participant in participants_df.iterrows():
    participants_df.at[i,classmates_column_name] = participants_df.drop(i).Roepnaam.sample(frac=1).str.cat(sep=', ')

In [33]:
participants_df["language"] = "nl"


In [34]:
participants_df

,Unnamed: 0,Grade,attribute_3 <Klas>,Roepnaam,Tussenv,Achternaam,Geboortedatum,Geslacht,token,Mentor,attribute_2 <Permission>,Persoonsgegevens,attribute_1 <Classmates>,language
0,NaN,1,A1g1,Jaro,NaN,NaN,NaN,NaN,1,Giovanni,no,NaN,"Hubert, Toon, John, Sanyogita, Loes",nl
1,NaN,1,A1g1,Loes,NaN,NaN,NaN,NaN,2,Giovanni,no,NaN,"John, Sanyogita, Jaro, Hubert, Toon",nl
2,NaN,1,A1g1,Toon,NaN,NaN,NaN,NaN,3,Giovanni,NaN,NaN,"Hubert, Jaro, John, Loes, Sanyogita",nl
3,NaN,1,A1g2,Sanyogita,NaN,NaN,NaN,NaN,4,Giovanni,NaN,NaN,"Loes, Hubert, John, Toon, Jaro",nl
4,NaN,1,A1g2,John,NaN,NaN,NaN,NaN,5,Wilbert,NaN,NaN,"Toon, Jaro, Loes, Hubert, Sanyogita",nl
5,NaN,1,A1g2,Hubert,NaN,NaN,NaN,NaN,6,Wilbert,NaN,NaN,"Jaro, John, Sanyogita, Loes, Toon",nl


In [35]:
participant_output_filename = participant_input_filename.with_stem(participant_input_filename.stem + "_preprocessed")
print(participant_output_filename)
participants_df.to_csv(participant_output_filename)

/home/jaro/NLeSC/QANS/qans-example-data/test-participant-tokens_preprocessed.csv
